<a href="https://colab.research.google.com/github/vikaspathak0911/19th-march-2025-fontend/blob/main/Judo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ultralytics
!pip install pytesseract


In [4]:
from ultralytics import YOLO
import cv2
import numpy as np
import pytesseract # <<< ADD THIS IMPORT


# --- 1. Load a pre-trained YOLOv8 SEGMENTATION model ---
# IMPORTANT: Use 'yolov8n-seg.pt' (or 'yolov8s-seg.pt', etc.) for masks!
person_segmentation_model = YOLO('yolov8n-seg.pt')

# --- 2. Load your custom-trained YOLOv8 model for TIMER DETECTION ---
# You need to train this model first and put its path here.
# For example: model_timer = YOLO('runs/detect/train_timer_dataset/weights/best.pt')
timer_detection_model = YOLO('/path/to/your/trained_timer_model/best.pt') # <<< REPLACE WITH YOUR PATH

# --- Configure Tesseract OCR ---
# Make sure Tesseract is installed and its path is correctly set if needed
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract' # Uncomment and set if Tesseract isn't in PATH
OCR_CONFIG = r'--oem 3 --psm 6 outputbase digits' # Optimize for digits

# --- Video Input ---
video_path = '/content/drive/My Drive/Japan vs South Korea .mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    print("Please check the file path and ensure the video file exists and GDrive is mounted.")
    exit()



100%|██████████| 6.74M/6.74M [00:00<00:00, 75.7MB/s]


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/trained_timer_model/best.pt'

In [ ]:
# --- Define Frame Range to Process (Optional, for testing) ---
start_frame_number = 900 # Example: Start from beginning
end_frame_number = 1500 # Example: Process up to frame 500

# Set the video's starting position (only if start_frame_number > 0)
if start_frame_number > 0:
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
    print(f"Set video playback to start from frame: {start_frame_number}")

# --- Video Output Setup ---
output_video_path = f'/content/drive/My Drive/judo_detection_output_color_3players_frames_{start_frame_number}_to_{end_frame_number}.mp4'

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 output
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# --- Define ROI (for persons, not timer if using YOLOv8 for timer) ---
# This ROI is for limiting person detection to the center, keep it as is.
center_roi_height_ratio = 0.7

print(f"Processing video: {video_path}")
print(f"Output will be saved to: {output_video_path}")
print(f"Original FPS: {fps}, Resolution: {frame_width}x{frame_height}")
print(f"Processing frames from {start_frame_number} to {end_frame_number}...")



In [ ]:
# --- Function to determine dominant clothing color ---
def get_dominant_color(roi_img, mask):
    """
    Analyzes pixels within the mask of the ROI image to determine dominant color.
    Assumes Judo-specific colors (white, blue).
    """
    if roi_img is None or mask is None or mask.sum() == 0:
        return "N/A"

    # Resize mask to match the cropped image's dimensions
    mask_resized = cv2.resize(mask, (roi_img.shape[1], roi_img.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Convert mask to boolean for direct indexing (0/1 values to True/False)
    mask_bool = mask_resized > 0

    # Now, use the boolean mask to get only the pixels belonging to the person
    person_pixels = roi_img[mask_bool]

    if len(person_pixels) == 0:
        return "N/A" # No pixels found within the mask

    # Convert to HSV color space for better color analysis
    # Keep the shape as (N, 1, 3) which is an image-like structure
    hsv_pixels = cv2.cvtColor(person_pixels.astype(np.uint8).reshape(-1, 1, 3), cv2.COLOR_BGR2HSV) # No need for the second reshape

    # Define common Judo gi color ranges in HSV (Hue: 0-179, Saturation/Value: 0-255)
    # Ensure bounds are 1D arrays of dtype=np.uint8
    white_lower = np.array([0, 0, 150], dtype=np.uint8)
    white_upper = np.array([179, 70, 255], dtype=np.uint8)

    blue_lower = np.array([90, 50, 50], dtype=np.uint8)
    blue_upper = np.array([130, 255, 255], dtype=np.uint8)

    black_lower = np.array([0, 0, 0], dtype=np.uint8)
    black_upper = np.array([179, 255, 60], dtype=np.uint8)

    # Count pixels falling into each range
    # Pass hsv_pixels in the (N, 1, 3) format
    white_pixels = np.sum(cv2.inRange(hsv_pixels, white_lower, white_upper))
    blue_pixels = np.sum(cv2.inRange(hsv_pixels, blue_lower, blue_upper))
    black_pixels = np.sum(cv2.inRange(hsv_pixels, black_lower, black_upper))

    # Determine dominant color based on pixel counts
    color_counts = {
        "White": white_pixels,
        "Blue": blue_pixels,
        "Black": black_pixels,
    }

    filtered_counts = {k: v for k, v in color_counts.items() if v > 0}

    if not filtered_counts:
        return "Other/Undefined"

    dominant_name = max(filtered_counts, key=filtered_counts.get)
    return dominant_name



In [ ]:
# --- Timer State Variables ---
last_timer_value = None
timer_is_running_flag = False
timer_detection_confidence_threshold = 0.5 # Confidence for YOLOv8 timer detection


In [ ]:
# --- Main Video Processing Loop ---
current_frame_number = start_frame_number
while True:
    ret, frame = cap.read()

    if not ret:
        print("End of video stream or error reading frame.")
        break

    if current_frame_number >= end_frame_number:
        print(f"Reached end_frame_number ({end_frame_number}). Stopping.")
        break

    if current_frame_number % int(fps) == 0:
        print(f"Processing frame {current_frame_number}...")

    frame_height, frame_width, _ = frame.shape

    # --- TIMER DETECTION LOGIC ---
    current_timer_value = None
    timer_detected_in_frame = False

    # Run inference for timer detection
    timer_results = timer_detection_model(frame, conf=timer_detection_confidence_threshold, verbose=False)

    if timer_results and timer_results[0].boxes and len(timer_results[0].boxes) > 0:
        # Assuming only one main timer, take the first detection (highest confidence)
        timer_box_data = timer_results[0].boxes[0]
        timer_confidence = timer_box_data.conf.cpu().numpy()[0] # Get confidence of the top timer detection

        if timer_confidence > timer_detection_confidence_threshold: # Check confidence
            tx1, ty1, tx2, ty2 = map(int, timer_box_data.xyxy[0].cpu().numpy())

            # Ensure the bounding box is within frame boundaries
            tx1, ty1, tx2, ty2 = max(0, tx1), max(0, ty1), min(frame.shape[1], tx2), min(frame.shape[0], ty2)

            timer_region = frame[ty1:ty2, tx1:tx2]

            if timer_region.size > 0: # Ensure region is not empty
                # Preprocessing for OCR
                gray_timer_region = cv2.cvtColor(timer_region, cv2.COLOR_BGR2GRAY)
                # Apply appropriate thresholding (Otsu is a good start)
                _, thresh_timer_region = cv2.threshold(gray_timer_region, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Further noise reduction/morphological operations might be needed depending on timer quality
                # For example:
                # kernel = np.ones((1,1), np.uint8)
                # thresh_timer_region = cv2.erode(thresh_timer_region, kernel, iterations=1)
                # thresh_timer_region = cv2.dilate(thresh_timer_region, kernel, iterations=1)

                try:
                    # Use Tesseract to read digits
                    text = pytesseract.image_to_string(thresh_timer_region, config=OCR_CONFIG)

                    # Clean and convert to integer (e.g., "01:23" -> 123, or just "123")
                    # You might need more sophisticated parsing depending on your timer format (e.g., MM:SS)
                    cleaned_text = ''.join(filter(str.isdigit, text))

                    if cleaned_text:
                        # If timer is MM:SS, convert to total seconds for comparison
                        if ':' in text: # Simple check for MM:SS format
                            parts = text.split(':')
                            if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                                current_timer_value = int(parts[0]) * 60 + int(parts[1])
                            else:
                                current_timer_value = int(cleaned_text) # Fallback if parsing fails
                        else:
                            current_timer_value = int(cleaned_text)

                        timer_detected_in_frame = True
                except Exception as e:
                    # print(f"OCR Error on timer region: {e}") # Uncomment for debugging OCR issues
                    pass # Keep current_timer_value as None if OCR fails
            # else:
            #     print("Detected timer region is empty after cropping, skipping OCR.")

    # Determine if timer is running based on value change
    if timer_detected_in_frame and last_timer_value is not None:
        # Check for any change in value to indicate it's running
        # You might want a more specific condition, e.g., current_timer_value < last_timer_value
        # if timer is counting down, or current_timer_value > last_timer_value if counting up.
        if abs(current_timer_value - last_timer_value) > 0:
            timer_is_running_flag = True
        else:
            timer_is_running_flag = False # Timer is static or paused
    elif timer_detected_in_frame and last_timer_value is None:
         # First frame with timer detected, can't determine running yet
         timer_is_running_flag = False
    else:
        timer_is_running_flag = False # No timer detected or OCR failed for multiple frames

    last_timer_value = current_timer_value # Update for next frame

    # --- Draw Timer Info for Visualization ---
    if timer_detected_in_frame:
        cv2.rectangle(frame, (tx1, ty1), (tx2, ty2), (255, 165, 0), 2) # Orange box for timer
        timer_status_text = f"Timer: {current_timer_value if current_timer_value is not None else 'N/A'}"
        cv2.putText(frame, timer_status_text, (tx1, ty1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 165, 0), 2)
        cv2.putText(frame, f"Running: {timer_is_running_flag}", (tx1, ty1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 165, 0), 2)


    # --- CONDITIONAL CLOTH COLOR DETECTION ---
    if timer_is_running_flag:
        print(f"Timer is running ({current_timer_value}). Proceeding with cloth color detection.")

        # Calculate the coordinates for the dynamic center ROI
        roi_w = int(frame_width * center_roi_width_ratio)
        roi_h = int(frame_height * center_roi_height_ratio)
        roi_x1 = (frame_width - roi_w) // 2
        roi_y1 = (frame_height - roi_h) // 2
        roi_x2 = roi_x1 + roi_w
        roi_y2 = roi_y1 + roi_h

        cv2.rectangle(frame, (roi_x1, roi_y1), (roi_x2, roi_y2), (255, 0, 0), 2)  # Draw person ROI


        # Use the person segmentation model

        results = model.predict(frame, conf=0.5, classes=[0], verbose=False)

        person_detections = []

        for r in results:
            if r.boxes is not None and len(r.boxes) > 0 and r.masks is not None and len(r.masks) > 0:
                boxes = r.boxes.xyxy.cpu().numpy()
                confidences = r.boxes.conf.cpu().numpy()
                class_ids = r.boxes.cls.cpu().numpy()
                masks = r.masks.data.cpu().numpy()

                for i in range(len(boxes)):
                    x1, y1, x2, y2 = map(int, boxes[i])
                    confidence = confidences[i]
                    class_id = int(class_ids[i])

                    if class_id == 0: # It's a person
                        center_x = (x1 + x2) // 2
                        center_y = (y1 + y2) // 2
                        bbox_area = (x2 - x1) * (y2 - y1)
                        is_in_roi = (roi_x1 <= center_x <= roi_x2 and
                                    roi_y1 <= center_y <= roi_y2)

                        if is_in_roi:
                            person_detections.append({
                                'box': (x1, y1, x2, y2),
                                'confidence': confidence,
                                'center': (center_x, center_y),
                                'area': bbox_area,
                                'mask': masks[i]
                            })

        selected_players = []
        num_players_to_detect = 3

        if len(person_detections) > 0:
            person_detections.sort(key=lambda p: p['area'], reverse=True)
            selected_players = person_detections[:num_players_to_detect]


        for player in selected_players:
            x1, y1, x2, y2 = player['box']
            confidence = player['confidence']
            player_mask = player['mask']

            crop_x1 = max(0, x1)
            crop_y1 = max(0, y1)
            crop_x2 = min(frame_width, x2)
            crop_y2 = min(frame_height, y2)

            cropped_frame_for_color = frame[crop_y1:crop_y2, crop_x1:crop_x2]

            dominant_color = get_dominant_color(cropped_frame_for_color, player_mask)

            if dominant_color == "Blue":
                bbox_draw_color = (255, 0, 0)
            elif dominant_color == "White":
                bbox_draw_color = (0, 255, 255)
            elif dominant_color == "Black":
                bbox_draw_color = (0, 0, 0)
            else:
                bbox_draw_color = (0, 0, 255)

            line_thickness = 3

            label = f"Player ({dominant_color}): {confidence:.2f}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), bbox_draw_color, line_thickness)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, bbox_draw_color, 2)

     else:
        print("Timer is NOT running. Skipping person and cloth color detection.")
        # You could display a message on the frame indicating no detection
        cv2.putText(frame, "Timer Not Running", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)




    out.write(frame)

    current_frame_number += 1

# --- Release resources ---
cap.release()
out.release()
print(f"\nFinished processing. Output video saved to: {output_video_path}")
